In [3]:
import os
from dotenv import load_dotenv, find_dotenv

env_file = load_dotenv(find_dotenv())

if not env_file:
    raise FileNotFoundError('.env')
    
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [56]:
from langchain_openai import OpenAI

llm_model = OpenAI()

In [4]:
from langchain_openai import ChatOpenAI

model = 'gpt-3.5-turbo-0125'

chat_model = ChatOpenAI(model=model)

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}!"
)

llm_model_prompt = prompt.format(
    adjective='curious',
    topic='BJP Politics'
)

chat = chat_model.invoke(llm_model_prompt)

In [7]:
print(chat.content)

One curious story about BJP politics is the case of Pragya Thakur, a controversial BJP candidate who was accused of being involved in a terrorist attack. Thakur, also known as Sadhvi Pragya, was accused of being one of the masterminds behind the 2008 Malegaon bombings in Maharashtra, which killed six people and injured over a hundred.

Despite facing serious charges, Thakur was fielded as the BJP candidate for the Bhopal constituency in the 2019 Lok Sabha elections. Her candidature sparked widespread controversy and criticism, with many questioning the party's decision to nominate someone with such a tainted past.

Thakur's campaign was also marred by controversial statements, including claiming that she was blessed by a divine power that killed her enemies, and referring to Mahatma Gandhi's assassin, Nathuram Godse, as a "patriot".

In a surprising turn of events, Thakur went on to win the election by a significant margin, defeating the incumbent Congress candidate. Her victory raised

In [49]:
from langchain_core.prompts import ChatPromptTemplate

# help(ChatMessagePromptTemplate)

chat_prompt_msgs = ChatPromptTemplate.from_messages(
    [
        ("system", "you are an expert {language} translator"),
        ("human", "translate anything for to {language}"),
        ("ai", "Sure!"),
        ("human", "{question}")
    ]
)


msgs = chat_prompt_msgs.format_messages(
    language='hindi',
    question='What the BJP tactics of politics?')
    # profession='Political Science',
    # topic='Politics',
    # usr_input='Tell me about the inception of BJP in politics'
# )

resp = chat_model.invoke(msgs)

In [50]:
resp.content

'भाजपा की राजनीति के तरीके क्या हैं?'

In [64]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a json formatted response by attaching an `anwser` key for the following {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | chat_model | json_parser

In [65]:
messages = {'question': 'which is the biggest country in the world!'}

resp = json_chain.invoke(messages)

In [66]:
resp

{'answer': 'Russia'}

In [77]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Politics(BaseModel):
    politics: str = Field(description='question about the political situation')
    country: str = Field(description='answer to specific country\'s political situation')

In [78]:
parser = JsonOutputParser(pydantic_object=Politics)

In [81]:
political_prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

custom_output_chain = political_prompt | chat_model | parser

In [82]:
custom_output_chain.invoke(dict(query='tell me politics of Pakistan'))

{'politics': 'question about the political situation', 'country': 'Pakistan'}